In [3]:
import pandas as pd
in_file = "./u12_2018_2014.csv"
u18_df = pd.read_csv(in_file,encoding="UTF-8")

In [4]:
#列の順番を整理する time,playerの列を削除
u18_df=u18_df.loc[:,['id','leagu_name','year','month','day','round','team_home','team_away','url','results_away','results_home','goal_away','goal_home']]
u18_df.head(12)
# u18_df.shape

,id,leagu_name,year,month,day,round,team_home,team_away,url,results_away,results_home,goal_away,goal_home
0,1,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,石井FC,大阪市ジュネッスFC,http://www.jfa.jp/match/japan_u12_football_cha...,4,0,"22分 木本 瑞希,28分 林 大晴,30分 増井 那月,32分 川口 遼己",None
1,2,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,アリーバFC,柏レイソル,http://www.jfa.jp/match/japan_u12_football_cha...,4,6,"31分 石井 僚,34分 石井 僚,37分 丸山 洸,38分 藤谷 温大","4分 佐久川 友杜,6分 相賀 天晴,11分 相賀 天晴,11分 佐久川 友杜,21分 廣瀬..."
2,31,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第2節,石井FC,アリーバFC,http://www.jfa.jp/match/japan_u12_football_cha...,2,0,"8分 相賀 天晴,17分 佐久川 友杜",None
3,53,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,27日,第3節,石井FC,柏レイソル,http://www.jfa.jp/match/japan_u12_football_cha...,3,1,"2分 尾崎 芯太朗,32分 藤谷 温大,38分 加藤 悠人",25分 板東 晋之介
4,32,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第2節,大阪市ジュネッスFC,柏レイソル,http://www.jfa.jp/match/japan_u12_football_cha...,2,1,"3分 丸山 洸,34分 丸山 洸",25分 増井 那月
5,54,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,27日,第3節,大阪市ジュネッスFC,アリーバFC,http://www.jfa.jp/match/japan_u12_football_cha...,0,3,None,"13分 増井 那月,31分 川口 遼己,40分 増井 那月"
6,3,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,Jフィールド津山SC,アビスパ福岡,http://www.jfa.jp/match/japan_u12_football_cha...,2,0,"23分 森部 絢,26分 森部 絢",None
7,29,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第2節,Jフィールド津山SC,兵庫FC,http://www.jfa.jp/match/japan_u12_football_cha...,0,1,None,10分 上山 哲平
8,30,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第2節,アビスパ福岡,FCフェルボール愛知,http://www.jfa.jp/match/japan_u12_football_cha...,0,1,None,40分 速水 仁
9,55,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,27日,第3節,Jフィールド津山SC,FCフェルボール愛知,http://www.jfa.jp/match/japan_u12_football_cha...,4,0,"6分 小浦 京之助,8分 明石 望来,9分 水巻 時飛,21分 三辻 元希",None


In [5]:
# スコアがNone(=未実施)の試合の行を削除
u18_df=u18_df[u18_df['results_away'] != "None"]
u18_df.shape

(455, 13)

In [6]:
# 年が変わるときの行(カラム名が入っている)を削除(一応...)
u18_df=u18_df[u18_df['results_away'] != "results_away"]
u18_df.shape

(451, 13)

In [7]:
# # yearに"2002"が入ってしまっているものの対処(未実装)
# u18_df=u18_df[u18_df['year'].str.match('.*,2002')!= "True"]
# u18_df.shape

In [8]:
#idの混同を避けるため、idとyearを合わせる
u18_df["merg_id"] = u18_df["year"].astype(str)+u18_df["id"].astype(str)
u18_df["merg_id"] = u18_df["merg_id"].astype(int)
u18_df.dtypes

id              object
leagu_name      object
year            object
month           object
day             object
round           object
team_home       object
team_away       object
url             object
results_away    object
results_home    object
goal_away       object
goal_home       object
merg_id          int32
dtype: object

In [9]:
# ここからaway

In [10]:
#awayチームの得点者とその試合のidのみを抜き出す
df_spr_away = pd.DataFrame(u18_df['goal_away'].str.split(',', expand=True))
df_spr_away["merg_id"] =u18_df["merg_id"]

In [11]:
# 先ほどの表から、1ゴール目のデータだけの表を作り、ゴールのない試合は削除
df_spr_away0 = pd.DataFrame({
    "goal":df_spr_away[0],
    "merg_id":df_spr_away["merg_id"]
})
df_spr_away0=df_spr_away0[df_spr_away0['goal'] != "None"] # df_spr_away0.dropna(inplace=True) はもともと"None"なので、使えない
df_spr_away0["count_away"]=int(1)
df_spr_away0.head(3)

,goal,merg_id,count_away
0,22分 木本 瑞希,20181,1
1,31分 石井 僚,20182,1
2,8分 相賀 天晴,201831,1


In [12]:
#2ゴール目以降のデータも追加
for repeat in range(1, ( (len(df_spr_away.columns)) -1) ):
    df_spr_away_n = pd.DataFrame({
    "goal":df_spr_away[repeat],
    "merg_id":df_spr_away["merg_id"],
    })
    df_spr_away_n.dropna(inplace=True)
    df_spr_away_n["count_away"]=int(1+repeat)
    df_spr_away0 = pd.concat([df_spr_away0,df_spr_away_n],sort=True,copy=False,ignore_index=True)

df_spr_away0

,count_away,goal,merg_id
0,1,22分 木本 瑞希,20181
1,1,31分 石井 僚,20182
2,1,8分 相賀 天晴,201831
3,1,2分 尾崎 芯太朗,201853
4,1,3分 丸山 洸,201832
5,1,23分 森部 絢,20183
6,1,6分 小浦 京之助,201855
7,1,27分 山内 愛翔,201828
8,1,4分 竹内 孝誠,201850
9,1,25分 大貫 天太郎,20187


In [13]:
#選手名と時間を分ける
df_spr_away0 = pd.concat([df_spr_away0,df_spr_away0["goal"].str.split("分",expand=True)],axis=1).drop("goal", axis=1)
df_spr_away0.rename(columns={0: 'time', 1: 'player'}, inplace=True)

# additional timeで別カラム
df_spr_away0 = pd.concat([df_spr_away0,df_spr_away0["time"].str.split("+",expand=True)],axis=1).drop("time", axis=1).fillna(-1)
df_spr_away0.rename(columns={0: 'n_time', 1: 'a_time'}, inplace=True)
df_spr_away0["n_time"] =df_spr_away0["n_time"].astype(int)
df_spr_away0["n_time"] =df_spr_away0["n_time"].astype(int)

# df_spr_away0["a_time"] =pd.to_numeric(df_spr_away0["a_time"], errors='coerce')

df_spr_away0

,count_away,merg_id,player,n_time,a_time
0,1,20181,木本 瑞希,22,-1
1,1,20182,石井 僚,31,-1
2,1,201831,相賀 天晴,8,-1
3,1,201853,尾崎 芯太朗,2,-1
4,1,201832,丸山 洸,3,-1
5,1,20183,森部 絢,23,-1
6,1,201855,小浦 京之助,6,-1
7,1,201828,山内 愛翔,27,-1
8,1,201850,竹内 孝誠,4,-1
9,1,20187,大貫 天太郎,25,-1


In [14]:
# 名前に「分」が入ってる人
def inc_hunn(df):
    try:
        df.loc[ (df[3] != -1 ), "player"] = df["player"].astype(str) + str("分") + df[2].astype(str)
        del df[3]
    except:
        try:
            df.loc[ (df[2] != -1 ), "player"] = df["player"].astype(str) + str("分") + df[2].astype(str)
            del df[2]
        except:
            pass
inc_hunn(df_spr_away0)

In [15]:
#ここからhome

In [16]:
#homeチームの得点者とその試合のidのみを抜き出す
df_spr_home = pd.DataFrame(u18_df['goal_home'].str.split(',', expand=True))
df_spr_home["merg_id"] =u18_df["merg_id"]

In [17]:
# 先ほどの表から、1ゴール目のデータだけの表を作り、ゴールのない試合は削除
df_spr_home0 = pd.DataFrame({
    "goal":df_spr_home[0],
    "merg_id":df_spr_home["merg_id"]
})
df_spr_home0=df_spr_home0[df_spr_home0['goal'] != "None"] # df_spr_home0.dropna(inplace=True) はもともと"None"なので、使えない
df_spr_home0["count_home"]=int(1)

#2ゴール目以降のデータも追加
for repeat in range(1, ( (len(df_spr_home.columns)) -1) ):
    df_spr_home_n = pd.DataFrame({
    "goal":df_spr_home[repeat],
    "merg_id":df_spr_home["merg_id"],
    })
    df_spr_home_n.dropna(inplace=True)
    df_spr_home_n["count_home"]=int(1+repeat)
    df_spr_home0 = pd.concat([df_spr_home0,df_spr_home_n],sort=True,copy=False,ignore_index=True)

df_spr_home0

,count_home,goal,merg_id
0,1,4分 佐久川 友杜,20182
1,1,25分 板東 晋之介,201853
2,1,25分 増井 那月,201832
3,1,13分 増井 那月,201854
4,1,10分 上山 哲平,201829
5,1,40分 速水 仁,201830
6,1,10分 速水 仁,201856
7,1,2分 香取 武,20185
8,1,6分 竹内 孝誠,20186
9,1,1分 田所 莉旺,201827


In [18]:
#選手名と時間を分ける
# df3 = pd.concat([df['local'], df['domain'].str.split('.', expand=True)], axis=1)
df_spr_home0 = pd.concat([df_spr_home0,df_spr_home0["goal"].str.split("分",expand=True)],axis=1).drop("goal", axis=1)
df_spr_home0.rename(columns={0: 'time', 1: 'player'}, inplace=True)

# additional timeで別カラム
df_spr_home0 = pd.concat([df_spr_home0,df_spr_home0["time"].str.split("+",expand=True)],axis=1).drop("time", axis=1).fillna(-1)
df_spr_home0.rename(columns={0: 'n_time', 1: 'a_time'}, inplace=True)
df_spr_home0["n_time"] =df_spr_home0["n_time"].astype(int)
df_spr_home0["a_time"] =df_spr_home0["a_time"].astype(int)

df_spr_home0

,count_home,merg_id,player,n_time,a_time
0,1,20182,佐久川 友杜,4,-1
1,1,201853,板東 晋之介,25,-1
2,1,201832,増井 那月,25,-1
3,1,201854,増井 那月,13,-1
4,1,201829,上山 哲平,10,-1
5,1,201830,速水 仁,40,-1
6,1,201856,速水 仁,10,-1
7,1,20185,香取 武,2,-1
8,1,20186,竹内 孝誠,6,-1
9,1,201827,田所 莉旺,1,-1


In [19]:
# 名前に「分」が入ってる人(いない場合はエラー、無視して次に進む)
inc_hunn(df_spr_home0)

In [20]:
#ここからまとめて

In [21]:
#homeとawayを合体 #並び変える（ソート）
df_merge = pd.concat([df_spr_away0,df_spr_home0],sort=True,copy=False,ignore_index=True)
# .sort_values(["id","n_time","a_time"]).reset_index(drop=True)
df_merged = pd.merge(df_merge,u18_df, on="merg_id").sort_values(["year","id","n_time","a_time"],ascending=[False,True,True,True]).reset_index(drop=True)

df_merged["differ"] =df_merged["merg_id"].diff().fillna(1)
df_merged

,a_time,count_away,count_home,merg_id,n_time,player,id,leagu_name,year,month,day,round,team_home,team_away,url,results_away,results_home,goal_away,goal_home,differ
0,-1,1.0,NaN,20181,22,木本 瑞希,1,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,石井FC,大阪市ジュネッスFC,http://www.jfa.jp/match/japan_u12_football_cha...,4,0,"22分 木本 瑞希,28分 林 大晴,30分 増井 那月,32分 川口 遼己",None,1.0
1,-1,2.0,NaN,20181,28,林 大晴,1,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,石井FC,大阪市ジュネッスFC,http://www.jfa.jp/match/japan_u12_football_cha...,4,0,"22分 木本 瑞希,28分 林 大晴,30分 増井 那月,32分 川口 遼己",None,0.0
2,-1,3.0,NaN,20181,30,増井 那月,1,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,石井FC,大阪市ジュネッスFC,http://www.jfa.jp/match/japan_u12_football_cha...,4,0,"22分 木本 瑞希,28分 林 大晴,30分 増井 那月,32分 川口 遼己",None,0.0
3,-1,4.0,NaN,20181,32,川口 遼己,1,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,石井FC,大阪市ジュネッスFC,http://www.jfa.jp/match/japan_u12_football_cha...,4,0,"22分 木本 瑞希,28分 林 大晴,30分 増井 那月,32分 川口 遼己",None,0.0
4,-1,1.0,NaN,201810,7,川村 紀吏,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,"7分 川村 紀吏,22分 川村 紀吏,30分 芹澤 飛勇,39分 矢島 礼偉","9分 渡辺 涼介,26分 小西 悠世,36分 峯野 倖",181629.0
5,-1,NaN,1.0,201810,9,渡辺 涼介,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,"7分 川村 紀吏,22分 川村 紀吏,30分 芹澤 飛勇,39分 矢島 礼偉","9分 渡辺 涼介,26分 小西 悠世,36分 峯野 倖",0.0
6,-1,2.0,NaN,201810,22,川村 紀吏,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,"7分 川村 紀吏,22分 川村 紀吏,30分 芹澤 飛勇,39分 矢島 礼偉","9分 渡辺 涼介,26分 小西 悠世,36分 峯野 倖",0.0
7,-1,NaN,2.0,201810,26,小西 悠世,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,"7分 川村 紀吏,22分 川村 紀吏,30分 芹澤 飛勇,39分 矢島 礼偉","9分 渡辺 涼介,26分 小西 悠世,36分 峯野 倖",0.0
8,-1,3.0,NaN,201810,30,芹澤 飛勇,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,"7分 川村 紀吏,22分 川村 紀吏,30分 芹澤 飛勇,39分 矢島 礼偉","9分 渡辺 涼介,26分 小西 悠世,36分 峯野 倖",0.0
9,-1,NaN,3.0,201810,36,峯野 倖,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,"7分 川村 紀吏,22分 川村 紀吏,30分 芹澤 飛勇,39分 矢島 礼偉","9分 渡辺 涼介,26分 小西 悠世,36分 峯野 倖",0.0


In [22]:
#idが変わる試合 かつ counthomeがnullのもの(=相手が先制しているので、counthomeは0になるべき)
df_merged.loc[(df_merged["differ"] != 0.0) &  (df_merged["count_home"].isnull() ), "count_home"] = 0
#awayも同様
df_merged.loc[(df_merged["differ"] != 0.0) &  (df_merged["count_away"].isnull() ), "count_away"] = 0

df_merged

,a_time,count_away,count_home,merg_id,n_time,player,id,leagu_name,year,month,day,round,team_home,team_away,url,results_away,results_home,goal_away,goal_home,differ
0,-1,1.0,0.0,20181,22,木本 瑞希,1,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,石井FC,大阪市ジュネッスFC,http://www.jfa.jp/match/japan_u12_football_cha...,4,0,"22分 木本 瑞希,28分 林 大晴,30分 増井 那月,32分 川口 遼己",None,1.0
1,-1,2.0,NaN,20181,28,林 大晴,1,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,石井FC,大阪市ジュネッスFC,http://www.jfa.jp/match/japan_u12_football_cha...,4,0,"22分 木本 瑞希,28分 林 大晴,30分 増井 那月,32分 川口 遼己",None,0.0
2,-1,3.0,NaN,20181,30,増井 那月,1,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,石井FC,大阪市ジュネッスFC,http://www.jfa.jp/match/japan_u12_football_cha...,4,0,"22分 木本 瑞希,28分 林 大晴,30分 増井 那月,32分 川口 遼己",None,0.0
3,-1,4.0,NaN,20181,32,川口 遼己,1,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,石井FC,大阪市ジュネッスFC,http://www.jfa.jp/match/japan_u12_football_cha...,4,0,"22分 木本 瑞希,28分 林 大晴,30分 増井 那月,32分 川口 遼己",None,0.0
4,-1,1.0,0.0,201810,7,川村 紀吏,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,"7分 川村 紀吏,22分 川村 紀吏,30分 芹澤 飛勇,39分 矢島 礼偉","9分 渡辺 涼介,26分 小西 悠世,36分 峯野 倖",181629.0
5,-1,NaN,1.0,201810,9,渡辺 涼介,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,"7分 川村 紀吏,22分 川村 紀吏,30分 芹澤 飛勇,39分 矢島 礼偉","9分 渡辺 涼介,26分 小西 悠世,36分 峯野 倖",0.0
6,-1,2.0,NaN,201810,22,川村 紀吏,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,"7分 川村 紀吏,22分 川村 紀吏,30分 芹澤 飛勇,39分 矢島 礼偉","9分 渡辺 涼介,26分 小西 悠世,36分 峯野 倖",0.0
7,-1,NaN,2.0,201810,26,小西 悠世,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,"7分 川村 紀吏,22分 川村 紀吏,30分 芹澤 飛勇,39分 矢島 礼偉","9分 渡辺 涼介,26分 小西 悠世,36分 峯野 倖",0.0
8,-1,3.0,NaN,201810,30,芹澤 飛勇,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,"7分 川村 紀吏,22分 川村 紀吏,30分 芹澤 飛勇,39分 矢島 礼偉","9分 渡辺 涼介,26分 小西 悠世,36分 峯野 倖",0.0
9,-1,NaN,3.0,201810,36,峯野 倖,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,"7分 川村 紀吏,22分 川村 紀吏,30分 芹澤 飛勇,39分 矢島 礼偉","9分 渡辺 涼介,26分 小西 悠世,36分 峯野 倖",0.0


In [23]:
#残りのNaNは、相手ゴール分なので、一つ上の行のデータで埋める
df_merged["count_away"]=df_merged["count_away"].fillna(method='ffill')
df_merged["count_home"]=df_merged["count_home"].fillna(method='ffill')
df_merged

,a_time,count_away,count_home,merg_id,n_time,player,id,leagu_name,year,month,day,round,team_home,team_away,url,results_away,results_home,goal_away,goal_home,differ
0,-1,1.0,0.0,20181,22,木本 瑞希,1,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,石井FC,大阪市ジュネッスFC,http://www.jfa.jp/match/japan_u12_football_cha...,4,0,"22分 木本 瑞希,28分 林 大晴,30分 増井 那月,32分 川口 遼己",None,1.0
1,-1,2.0,0.0,20181,28,林 大晴,1,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,石井FC,大阪市ジュネッスFC,http://www.jfa.jp/match/japan_u12_football_cha...,4,0,"22分 木本 瑞希,28分 林 大晴,30分 増井 那月,32分 川口 遼己",None,0.0
2,-1,3.0,0.0,20181,30,増井 那月,1,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,石井FC,大阪市ジュネッスFC,http://www.jfa.jp/match/japan_u12_football_cha...,4,0,"22分 木本 瑞希,28分 林 大晴,30分 増井 那月,32分 川口 遼己",None,0.0
3,-1,4.0,0.0,20181,32,川口 遼己,1,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,石井FC,大阪市ジュネッスFC,http://www.jfa.jp/match/japan_u12_football_cha...,4,0,"22分 木本 瑞希,28分 林 大晴,30分 増井 那月,32分 川口 遼己",None,0.0
4,-1,1.0,0.0,201810,7,川村 紀吏,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,"7分 川村 紀吏,22分 川村 紀吏,30分 芹澤 飛勇,39分 矢島 礼偉","9分 渡辺 涼介,26分 小西 悠世,36分 峯野 倖",181629.0
5,-1,1.0,1.0,201810,9,渡辺 涼介,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,"7分 川村 紀吏,22分 川村 紀吏,30分 芹澤 飛勇,39分 矢島 礼偉","9分 渡辺 涼介,26分 小西 悠世,36分 峯野 倖",0.0
6,-1,2.0,1.0,201810,22,川村 紀吏,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,"7分 川村 紀吏,22分 川村 紀吏,30分 芹澤 飛勇,39分 矢島 礼偉","9分 渡辺 涼介,26分 小西 悠世,36分 峯野 倖",0.0
7,-1,2.0,2.0,201810,26,小西 悠世,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,"7分 川村 紀吏,22分 川村 紀吏,30分 芹澤 飛勇,39分 矢島 礼偉","9分 渡辺 涼介,26分 小西 悠世,36分 峯野 倖",0.0
8,-1,3.0,2.0,201810,30,芹澤 飛勇,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,"7分 川村 紀吏,22分 川村 紀吏,30分 芹澤 飛勇,39分 矢島 礼偉","9分 渡辺 涼介,26分 小西 悠世,36分 峯野 倖",0.0
9,-1,3.0,3.0,201810,36,峯野 倖,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,"7分 川村 紀吏,22分 川村 紀吏,30分 芹澤 飛勇,39分 矢島 礼偉","9分 渡辺 涼介,26分 小西 悠世,36分 峯野 倖",0.0


In [24]:
#試合情報を結合
# df_merged = pd.merge(df_merge,u18_df, on="id").sort_values("year",ascending=False)
df_merged["goal_away"]=df_merged["count_away"].astype(int)
df_merged["goal_home"]=df_merged["count_home"].astype(int)
# del df_merged["count_away"]
# del df_merged["count_home"]
df_merged

,a_time,count_away,count_home,merg_id,n_time,player,id,leagu_name,year,month,day,round,team_home,team_away,url,results_away,results_home,goal_away,goal_home,differ
0,-1,1.0,0.0,20181,22,木本 瑞希,1,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,石井FC,大阪市ジュネッスFC,http://www.jfa.jp/match/japan_u12_football_cha...,4,0,1,0,1.0
1,-1,2.0,0.0,20181,28,林 大晴,1,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,石井FC,大阪市ジュネッスFC,http://www.jfa.jp/match/japan_u12_football_cha...,4,0,2,0,0.0
2,-1,3.0,0.0,20181,30,増井 那月,1,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,石井FC,大阪市ジュネッスFC,http://www.jfa.jp/match/japan_u12_football_cha...,4,0,3,0,0.0
3,-1,4.0,0.0,20181,32,川口 遼己,1,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,石井FC,大阪市ジュネッスFC,http://www.jfa.jp/match/japan_u12_football_cha...,4,0,4,0,0.0
4,-1,1.0,0.0,201810,7,川村 紀吏,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,1,0,181629.0
5,-1,1.0,1.0,201810,9,渡辺 涼介,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,1,1,0.0
6,-1,2.0,1.0,201810,22,川村 紀吏,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,2,1,0.0
7,-1,2.0,2.0,201810,26,小西 悠世,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,2,2,0.0
8,-1,3.0,2.0,201810,30,芹澤 飛勇,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,3,2,0.0
9,-1,3.0,3.0,201810,36,峯野 倖,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,3,3,0.0


In [25]:
#additional timeをプラス表記に
df_merged["time"] = df_merged["n_time"]
df_merged.loc[ (df_merged["a_time"] !=-1 ), "time"] = df_merged["n_time"].astype(str) + str("+") + df_merged["a_time"].astype(str)

df_merged=df_merged.loc[:,['id','leagu_name','year','month','day','round','team_home','team_away','url','results_away','results_home','goal_away','goal_home','time','player']]
df_merged

,id,leagu_name,year,month,day,round,team_home,team_away,url,results_away,results_home,goal_away,goal_home,time,player
0,1,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,石井FC,大阪市ジュネッスFC,http://www.jfa.jp/match/japan_u12_football_cha...,4,0,1,0,22,木本 瑞希
1,1,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,石井FC,大阪市ジュネッスFC,http://www.jfa.jp/match/japan_u12_football_cha...,4,0,2,0,28,林 大晴
2,1,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,石井FC,大阪市ジュネッスFC,http://www.jfa.jp/match/japan_u12_football_cha...,4,0,3,0,30,増井 那月
3,1,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,石井FC,大阪市ジュネッスFC,http://www.jfa.jp/match/japan_u12_football_cha...,4,0,4,0,32,川口 遼己
4,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,1,0,7,川村 紀吏
5,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,1,1,9,渡辺 涼介
6,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,2,1,22,川村 紀吏
7,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,2,2,26,小西 悠世
8,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,3,2,30,芹澤 飛勇
9,10,JFA 第42回全日本U-12サッカー選手権大会,2018,12月,26日,第1節,大山田SSS,鹿島アントラーズつくば,http://www.jfa.jp/match/japan_u12_football_cha...,4,3,3,3,36,峯野 倖


In [26]:
# 出力CSVの名前
from pathlib import Path
newname = Path(in_file).stem
print (newname)

u12_2018_2014


In [27]:
# タイムスタンプ
import os
from datetime import datetime
t = os.path.getmtime(in_file)
# エポック秒をdatetimeに変換
dt = datetime.fromtimestamp(t)
datadate = dt.strftime('%Y%m%d%H%M')
print (datadate)

201910311544


In [28]:
# 出力
df_merged.to_csv("./adjust_" + newname + "_" + datadate + ".csv", 
          index=False   # 行インデックスを削除
         )

In [30]:
import subprocess
subprocess.run(['jupyter', 'nbconvert', '--to', 'python', 'dataformat.ipynb'])

CompletedProcess(args=['jupyter', 'nbconvert', '--to', 'python', 'dataformat.ipynb'], returncode=0)